# **DIAGNOSTICS**
> **Copy, Paste**, and **Run** the code below **optimization model** when you do not get an OPTIMAL SOLUTION

In [ ]:
print("="*80)
print("INFEASIBILITY DIAGNOSTIC")
print("="*80)

# Load data
df_faults = pd.read_csv('../dataset/ecg_faults_dataset.csv')

num_faults = len(df_faults)
num_groups = 5
Q = 3
H = 8
theta = 1.5

travel_time = df_faults['Travel_time_hours'].tolist()
repair_time = df_faults['Repair_time_hours'].tolist()
zones = df_faults['Zone'].tolist()

print(f"\n1.  CAPACITY CHECK:")
print(f"   Total capacity: {num_groups} groups × {Q} faults = {num_groups * Q}")
print(f"   Total faults: {num_faults}")
if num_groups * Q >= num_faults:
    print(f"   ✅ Capacity is sufficient")
else:
    print(f"   ❌ INSUFFICIENT CAPACITY!")

print(f"\n2.  TIME FEASIBILITY CHECK:")
print(f"   Checking if individual faults fit in {H}-hour shift:")

df_faults['Total_time'] = df_faults['Travel_time_hours'] + df_faults['Repair_time_hours']

for idx, row in df_faults.iterrows():
    total = row['Total_time']
    status = "✅" if total <= H else "❌"
    print(f"   {status} {row['Fault_ID']}: {total:.3f}h (Travel: {row['Travel_time_hours']:.3f}h + Repair: {row['Repair_time_hours']:.3f}h)")

max_time = df_faults['Total_time'].max()
if max_time > H:
    print(f"\n   ❌ PROBLEM FOUND: Some faults exceed shift duration!")
    print(f"   Longest fault: {max_time:.3f} hours > {H} hours")
else:
    print(f"\n   ✅ All individual faults fit within shift")

print(f"\n3.  WORKLOAD DISTRIBUTION CHECK:")
print(f"   Total work time needed: {df_faults['Total_time'].sum():.3f} hours")
print(f"   Available work time: {num_groups} groups × {H} hours = {num_groups * H} hours")
print(f"   Average per group: {df_faults['Total_time'].sum() / num_groups:.3f} hours")

if df_faults['Total_time'].sum() > num_groups * H:
    print(f"   ❌ PROBLEM: Total work exceeds available time!")
else:
    print(f"   ✅ Total work fits within available time")

print(f"\n4.  EQUITY CONSTRAINT CHECK:")
near_faults = df_faults[df_faults['Zone'] == 'Near']
far_faults = df_faults[df_faults['Zone'] == 'Far']

if len(near_faults) > 0 and len(far_faults) > 0:
    avg_near = near_faults['Travel_time_hours'].mean()
    avg_far = far_faults['Travel_time_hours'].mean()
    ratio = avg_far / avg_near
    
    print(f"   Near zone avg travel: {avg_near:.3f}h")
    print(f"   Far zone avg travel: {avg_far:.3f}h")
    print(f"   Current ratio: {ratio:.2f}")
    print(f"   Equity constraint: ≤ {theta}")
    
    if ratio <= theta:
        print(f"   ✅ Equity constraint is naturally satisfied")
    else:
        print(f"   ⚠️  Equity constraint may be restrictive")

print(f"\n5.  DETAILED FAULT ANALYSIS:")
print("\n" + df_faults[['Fault_ID', 'Town', 'Zone', 'Fault_type', 'Travel_time_hours', 
                         'Repair_time_hours', 'Total_time']].to_string(index=False))

print(f"\n6.  CHECKING FOR IMPOSSIBLE COMBINATIONS:")
# Check if we can fit 3 faults in any group
df_sorted = df_faults.sort_values('Total_time')
print(f"\n   Shortest 3 faults combined:")
shortest_3 = df_sorted.head(3)
total_shortest_3 = shortest_3['Total_time'].sum()
print(shortest_3[['Fault_ID', 'Town', 'Total_time']].to_string(index=False))
print(f"   Total: {total_shortest_3:.3f} hours")

if total_shortest_3 <= H:
    print(f"   ✅ At least 3 faults can fit in one group")
else:
    print(f"   ❌ PROBLEM: Even shortest 3 faults exceed {H} hours!")

print("\n" + "="*80)
print("DIAGNOSIS COMPLETE")
print("="*80)